<a href="https://colab.research.google.com/github/mcgmed/Tensorflow/blob/main/Detecting-Features-in-Images/Horse_or_Human_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import urllib.request
import zipfile

training_url = "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip"
training_file_name = "horse-or-human.zip"
training_dir = 'horse-or-human/training/'
urllib.request.urlretrieve(training_url, training_file_name)

zip_ref = zipfile.ZipFile(training_file_name, 'r')
zip_ref.extractall(training_dir)
zip_ref.close()

In [2]:
validation_url = "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip"
validation_file_name = "validation-horse-or-human.zip"
validation_dir = 'horse-or-human/validation/'
urllib.request.urlretrieve(validation_url, validation_file_name)

zip_ref = zipfile.ZipFile(validation_file_name, 'r')
zip_ref.extractall(validation_dir)
zip_ref.close()

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory(
        training_dir,
        target_size=(300, 300),
        class_mode='binary')

validation_datagen = ImageDataGenerator(rescale=1/255)

validation_generator = train_datagen.flow_from_directory(
        validation_dir,
        target_size=(300, 300),
        class_mode='binary')

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


In [4]:
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300,300,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=RMSprop(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 298, 298, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 149, 149, 16)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 147, 147, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 73, 73, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 71, 71, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 35, 35, 64)       0

In [5]:
history = model.fit(train_generator, epochs=15, validation_data=validation_generator)

Epoch 1/15
33/33 [==============================] - 18s 250ms/step - loss: 0.7403 - accuracy: 0.7488 - val_loss: 1.0428 - val_accuracy: 0.8320
Epoch 2/15
33/33 [==============================] - 8s 243ms/step - loss: 0.2100 - accuracy: 0.9124 - val_loss: 1.1875 - val_accuracy: 0.8164
Epoch 3/15
33/33 [==============================] - 8s 243ms/step - loss: 0.2422 - accuracy: 0.9309 - val_loss: 1.5797 - val_accuracy: 0.8008
Epoch 4/15
33/33 [==============================] - 8s 241ms/step - loss: 0.0725 - accuracy: 0.9776 - val_loss: 2.1428 - val_accuracy: 0.8086
Epoch 5/15
33/33 [==============================] - 8s 240ms/step - loss: 0.1092 - accuracy: 0.9718 - val_loss: 2.4287 - val_accuracy: 0.7617
Epoch 6/15
33/33 [==============================] - 8s 245ms/step - loss: 0.2146 - accuracy: 0.9825 - val_loss: 1.5199 - val_accuracy: 0.7930
Epoch 7/15
33/33 [==============================] - 9s 266ms/step - loss: 0.0461 - accuracy: 0.9873 - val_loss: 1.4562 - val_accuracy: 0.8398
Epoch

### **Testing**

In [19]:
import numpy as np
from google.colab import files
from tensorflow.keras.preprocessing import image

uploaded = files.upload()

Saving 0x0-horses-the-wings-of-mankind-1527015927739.jpg to 0x0-horses-the-wings-of-mankind-1527015927739 (3).jpg
Saving gelding-bay-coat.webp to gelding-bay-coat (3).webp
Saving istockphoto-1365223878-170667a.jpg to istockphoto-1365223878-170667a (3).jpg


In [20]:
uploaded.keys()

dict_keys(['0x0-horses-the-wings-of-mankind-1527015927739.jpg', 'gelding-bay-coat.webp', 'istockphoto-1365223878-170667a.jpg'])

In [23]:
for fn in uploaded.keys():
  path = '/content/' + fn
  img = image.load_img(path, target_size=(300, 300))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  image_tensor = np.vstack([x])
  classes = model.predict(image_tensor)
  print(classes)
  if classes[0]>0.5:
    print(fn + " is a human")
  else:
    print(fn + " is a horse")

1/1 [==============================] - 0s 17ms/step
[[0.]]
0x0-horses-the-wings-of-mankind-1527015927739.jpg is a horse
1/1 [==============================] - 0s 17ms/step
[[0.]]
gelding-bay-coat.webp is a horse
1/1 [==============================] - 0s 15ms/step
[[1.]]
istockphoto-1365223878-170667a.jpg is a human
